In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def split_real_im(c): 
    c = complex(c)
    return pd.Series([c.real,c.imag])

In [3]:
# import input and target data 
# all_input = pd.read_csv("../Samples_RTS24/oT_Result_NN_Input_RTS24.csv")
# all_output = pd.read_csv("../Samples_RTS24/oT_Result_NN_Output_RTS24.csv")

# all_input = pd.read_csv("../../Samples_RTS24_ACOPF/oT_Result_NN_Input_RTS24.csv")
# all_output = pd.read_csv("../../Samples_RTS24_ACOPF/oT_Result_NN_Output_RTS24.csv")

all_input = pd.read_csv("../10.StarNet/RTS24/ClusterResults/oT_Result_NN_Input_RTS24.csv")
all_output = pd.read_csv("../10.StarNet/RTS24/ClusterResults/oT_Result_NN_Output_RTS24.csv")

In [4]:
# timestamps = np.unique(all_input.LoadLevel)
# variables = np.sort(np.unique(all_input.Variable))
# executions = np.unique(all_input.Execution)
# periods = np.unique(all_input.Period)
# scenarios = np.unique(all_input.Scenario)

executions = np.unique(all_input.Execution)

variables_o =  np.unique(all_output.Variable)

In [5]:
variables = np.sort(np.unique(all_input.Variable))

In [6]:
timestamps = np.unique(all_input.LoadLevel)

In [7]:
executions = np.unique(all_input.Execution)

In [17]:
#Select only the relevant variables, for which the appropriate mapping file is read: 
generation_types = pd.read_csv("../Data/Samples_RTS24_ACOPF/oT_Data_Generation_Technology_RTS24.csv")
techs_kept = ["Hydro", "Solar","Wind"]
variables_to_drop_i = np.unique(generation_types[~generation_types.Technology.isin(techs_kept)].Unit)
variables_to_keep_i = [v for v in variables if v not in(variables_to_drop_i)]

variables_to_keep_o = ["vTotalCCost", "vTotalECost","vTotalGCost","vTotalRCost"]

In [20]:
variables_to_keep_i

['101_PV_1',
 '101_PV_2',
 '101_PV_3',
 '101_PV_4',
 '102_PV_1',
 '102_PV_2',
 '103_PV_1',
 '104_PV_1',
 '113_PV_1',
 '118_RTPV_1',
 '118_RTPV_10',
 '118_RTPV_2',
 '118_RTPV_3',
 '118_RTPV_4',
 '118_RTPV_5',
 '118_RTPV_6',
 '118_RTPV_7',
 '118_RTPV_8',
 '118_RTPV_9',
 '119_PV_1',
 '122_HYDRO_1',
 '122_HYDRO_2',
 '122_HYDRO_3',
 '122_HYDRO_4',
 '122_HYDRO_5',
 '122_HYDRO_6',
 '122_WIND_1',
 'N_101',
 'N_102',
 'N_103',
 'N_104',
 'N_105',
 'N_106',
 'N_107',
 'N_108',
 'N_109',
 'N_110',
 'N_111',
 'N_112',
 'N_113',
 'N_114',
 'N_115',
 'N_116',
 'N_117',
 'N_118',
 'N_119',
 'N_120',
 'N_121',
 'N_122',
 'N_123',
 'N_124',
 'Node_10_Node_1',
 'Node_10_Node_10',
 'Node_10_Node_11',
 'Node_10_Node_12',
 'Node_10_Node_13',
 'Node_10_Node_14',
 'Node_10_Node_15',
 'Node_10_Node_16',
 'Node_10_Node_17',
 'Node_10_Node_18',
 'Node_10_Node_19',
 'Node_10_Node_2',
 'Node_10_Node_20',
 'Node_10_Node_21',
 'Node_10_Node_22',
 'Node_10_Node_23',
 'Node_10_Node_24',
 'Node_10_Node_3',
 'Node_10_N

In [9]:
len(timestamps),len(variables),len(executions),len(periods),len(scenarios)

NameError: name 'periods' is not defined

In [21]:
folder = "RTS24_AC_12w_ext_o"
# per = periods[0]
# sc = scenarios[0]
per = 2030
sc = "sc01"

for exe in executions[0:1]: 
    variables_to_keep = variables
    # variables_to_keep_o = np.sort(np.unique(all_output.Variable))[[0,1,3,4]]
    variables_to_keep_o = ["vTotalCCost","vTotalECost","vTotalGCost","vTotalRCost"]

    #Filter the main frame on desired parameters: 
    f_i = (all_input.LoadLevel.isin(timestamps))& (all_input.Execution == exe ) & (all_input.Period == per )& (all_input.Scenario == sc)& (all_input.Variable.isin(variables_to_keep_i))
    input_f = pd.DataFrame(all_input[f_i])

    f_o = (all_output.LoadLevel.isin(timestamps))& (all_output.Execution == exe ) & (all_output.Period == per )& (all_output.Scenario == sc)& (all_output.Variable.isin(variables_to_keep_o))
    output_f = pd.DataFrame(all_output[f_o])

    # #And then add to new frame the two parts of the complex numbers: 
    # input_f[['Value_R', 'Value_I']] = input_f["Value"].apply(split_real_im)

    #Pivot 
    # input_f_p = input_f.pivot(index = "LoadLevel",columns=["Variable"],values = ["Value_R","Value_I"])
    # input_f_p = input_f.pivot(index = ["LoadLevel","Dataset"],columns=["Variable"],values = "Value")
    real_in_p = input_f[input_f.Dataset != "MatrixYImag"].pivot(index = ["LoadLevel"],columns=["Variable"],values = "Value")

    imag_in_p = input_f[input_f.Dataset == "MatrixYImag"].pivot(index = ["LoadLevel"],columns=["Variable"],values = "Value").add_suffix("_I")

    input_f_p = pd.concat([real_in_p,imag_in_p],axis=1).reindex(sorted(pd.concat([real_in_p,imag_in_p],axis=1).columns), axis=1)

    output_f_p = output_f.pivot(index = "LoadLevel",columns=["Variable"],values = "Value")

    #Write to separate csv file
    input_f_p.to_csv(f"../Data/{folder}/input_f_{sc}_{exe}_{per}.csv")
    output_f_p.to_csv(f"../Data/{folder}/output_f_{sc}_{exe}_{per}.csv")

In [17]:
#Select a certain scenario and execution
folder_to_write = "RTS_24_AC_12w"
#exe = executions[5]
per = periods[0]
sc = scenarios[0]
for exe in executions:

    #Filter the main frame on desired parameters: 
    f_i = (all_input.LoadLevel.isin(timestamps))& (all_input.Execution == exe ) & (all_input.Period == per )& (all_input.Scenario == sc)& (all_input.Variable.isin(variables_to_keep_i))
    input_f = pd.DataFrame(all_input[f_i])

    f_o = (all_output.LoadLevel.isin(timestamps))& (all_output.Execution == exe ) & (all_output.Period == per )& (all_output.Scenario == sc)& (all_output.Variable.isin(variables_to_keep_o))
    output_f = pd.DataFrame(all_output[f_o])

    #And then add to new frame the two parts of the complex numbers: 
    input_f[['Value_R', 'Value_I']] = input_f["Value"].apply(split_real_im)

    #Pivot 
    input_f_p = input_f.pivot(index = "LoadLevel",columns=["Variable"],values = ["Value_R","Value_I"])
    output_f_p = output_f.pivot(index = "LoadLevel",columns=["Variable","Dataset"],values = ["Value"])

    #Write to separate csv file
    input_f_p.to_csv(f"Data/{folder_to_write}/input_f_{sc}_{exe}_{per}.csv")
    output_f_p.to_csv(f"Data/{folder_to_write}/output_f_{sc}_{exe}_{per}.csv")